In [8]:
import pandas as pd
import urllib.request
import time
import datetime
import os

In [4]:
apikey = "ALPHACANTAGEKEY"
root = "/data/S&P500 Data/" # download location of data from Alpha Vantage
output = "/data/S&P500 Data/" # output of joining download files

In [6]:
# List of the 500 companies
pathSP500 = "/data/S&P500 Data/SP500.csv"
dfSP500 = pd.read_csv(pathSP500)

In [7]:
# Loop through and download the prices
for ticker in dfSP500["Ticker symbol"]:
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}&datatype=csv".format(ticker, apikey)
    pathDataFiles = "{}{}.csv".format(root, ticker)
    web = urllib.request.URLopener()
    web.retrieve(url, pathDataFiles)
    time.sleep(15)  # sleep for 15 seconds; Alpha Vantage only allows 5 calls per minute

In [9]:
# Lets take the data and join it into one table/file
dates = []
numDays = 365*3
for i in range(0,numDays):
    dates.append([(datetime.datetime(2018,9,11) - datetime.timedelta(days=365*3-i)).date(),0])
dfDates = pd.DataFrame(dates,columns=["Date","adjusted_close"])

df = dfDates
for fldr, subdirs, files in os.walk(root):
    for file in files:
        ticker = file.replace(".csv","")
        pathTicker = fldr.replace("\\","/") + "/" + file
        dfTicker = pd.read_csv(pathTicker, parse_dates=['timestamp'])
        dfTicker['timestamp'] = dfTicker['timestamp'].dt.date
        dfTicker = dfTicker.rename(index=str, columns={"timestamp":"Date"})
        dfTicker = dfTicker[['Date','adjusted_close']]
        df = df.merge(dfTicker, how='left', on=['Date'], suffixes=['','_{}'.format(ticker)])
        
df.to_csv(output + "/all2.csv")